# ADVANCED INTERACTIVE DATA VISUALIZATION

Now that we understand how Holoviews & Panel work and what we can do with them, let's go wild and throw in everything we can think of!

As always, feel free to send me an email to Alfonso Acosta Gonçalves (a.acostagoncalves@unsw.edu.au) with any questions, suggestions or comments.

# Imports & Data Loading

The code must be run on **Kernel 3-19.10**, which is the only one where panel & holoviews have been installed.

In [ ]:
%matplotlib inline

import calendar
import cartopy
import cartopy.crs as ccrs
import cmocean.cm as cmo
import holoviews as hv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import panel as pn
import xarray as xr

# Disable warning messages
import warnings
warnings.filterwarnings('ignore')

from bisect import bisect_left
from bokeh.palettes import brewer
from matplotlib import colors
from palettable.cartocolors.qualitative import Prism_8

# Enable panels & the bokeh back-end for Holoviews
pn.extension()
hv.extension('bokeh')

Functions' definitions:

In [ ]:
def closest_lat(sel_lat):
    """
    Returns closest latitude to sel_lat in yt coordinates (assumes dsm.yt_ocean is sorted)
    If two numbers are equally close, return the smallest.
    
    Parameters:
    -----------
        sel_lat: float with the selected latitude
    """
    pos = bisect_left(dsm.yt_ocean, sel_lat)
    if pos == 0:
        return dsm.yt_ocean[0].values
    if pos == len(dsm.yt_ocean):
        return dsm.yt_ocean[-1].values
    before = dsm.yt_ocean[pos - 1].values
    after = dsm.yt_ocean[pos].values
    if after - sel_lat < sel_lat - before:
       return after
    else:
       return before

In [ ]:
def closest_lon(sel_lon):
    """
    Returns closest longitude to sel_lon in xt coordinates (assumes dsm.xt_ocean is sorted)
    If two numbers are equally close, return the smallest.
    
    Parameters:
    -----------
        sel_lon: float with the selected longitude
    """
    pos = bisect_left(dsm.xt_ocean, sel_lon)
    if pos == 0:
        return dsm.xt_ocean[0].values
    if pos == len(dsm.xt_ocean):
        return dsm.xt_ocean[-1].values
    before = dsm.xt_ocean[pos - 1].values
    after = dsm.xt_ocean[pos].values
    if after - sel_lon < sel_lon - before:
       return after
    else:
       return before

In [ ]:
def closest_lon_u(sel_lon):
    """
    Returns closest longitude to sel_lon in xu coordinates (assumes dso.xu_ocean is sorted)
    If two numbers are equally close, return the smallest.
    
    Parameters:
    -----------
        sel_lon: float with the selected longitude
    """
    pos = bisect_left(dso.xu_ocean, sel_lon)
    if pos == 0:
        return dso.xu_ocean[0].values
    if pos == len(dso.xu_ocean):
        return dso.xu_ocean[-1].values
    before = dso.xu_ocean[pos - 1].values
    after = dso.xu_ocean[pos].values
    if after - sel_lon < sel_lon - before:
       return after
    else:
       return before

In [ ]:
def set_inactive_tool(plot, element): 
    """
    For Bokeh plots, deactivates the Hover tool on initial rendering
    """
    plot.state.toolbar.active_inspect = None 

Data loading:

In [ ]:
# Open files
dsm = xr.open_dataset('/home/561/ag9076/payu/1deg_maru/archive/output030/ocean/ocean_month.nc')
dso = xr.open_dataset('/home/561/ag9076/payu/1deg_maru/archive/output030/ocean/ocean.nc')

dsm_c = xr.open_dataset('/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_ryf9091_spinup1_B1/output051/ocean/ocean_month.nc')
dso_c = xr.open_dataset('/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_ryf9091_spinup1_B1/output051/ocean/ocean.nc')

topog = xr.open_dataset('/g/data4/ik11/inputs/access-om2/input_rc/mom_1deg/topog.nc')

# Rename & copy coordinates from one dataset to another so HoloViews knows which dims/coords are exactly the same
topog = topog.rename({'nx':'xt_ocean', 'ny':'yt_ocean'})
topog.coords['xt_ocean'] = dsm.coords['xt_ocean']
topog.coords['yt_ocean'] = dsm.coords['yt_ocean']

# 1. MLS, SSH & a TIME-SERIES FOR GOOD MEASURE

Now that we have all our building blocks, let's think what to do with them...

Those colorbars changing range automatically are not always the best idea, but setting the colorbar range "in stone" doesn't work either, because we may loose definition & nuances. Why not use a range slider to control the colorbar range?

And while we're at it, instead of looking at the MLD from above, couldn't we do one of those cross-sections to look at it from the side?

And it would be interesting to look at the SSH at the same time, why limit ourselves to just one cross-section?

In [ ]:
# MLD through time, averaged per month
ds_mld = dsm.mld.loc[:,-78:-45,:]
ds_mld = ds_mld.groupby('time.month').mean('time')
# Ctrl MLD through time, averaged per month
ds_mld_c = dsm_c.mld.loc[:,-78:-45,:]
ds_mld_c = ds_mld_c.groupby('time.month').mean('time')

# Bathymetry of the experiment
bathymetry = topog.depth.loc[-78:-45, :]

# Sea-Level through time, averaged per month
ds_sealevel = dsm.sea_level.loc[:,-78:-45,:]
ds_sealevel = ds_sealevel.groupby('time.month').mean('time')
ds_sealevel_c = dsm_c.sea_level.loc[:,-78:-45,:]
ds_sealevel_c = ds_sealevel_c.groupby('time.month').mean('time')

In [ ]:
hv.extension('bokeh')

colorbar_range_mld = (0, ds_mld.max().values.max()) # Get the MLD max value
cmap_mld = "cmo.deep_r"
initial_lon = -68.5

# Stream to follow the pointer over the upper plot
posx_mld = hv.streams.PointerX()

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS

# Widget to select the MONTH
month_slider_mld = pn.widgets.DiscreteSlider(name='Month', options=ds_mld.month.values.tolist(), 
                                             value=ds_mld.month.values[0], width=500)

# Widget to select the COLORBAR RANGE
colorbar_range_slider_mld = pn.widgets.RangeSlider(name='Colorbar Range', start=0, 
                                                   end=colorbar_range_mld[1],
                                                   value=colorbar_range_mld, step=100, width=500)

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS' & STREAMS' LINKS
# Watch for changes on the widgets' values to update the plots

@pn.depends(sel_month=month_slider_mld,
            col_range=colorbar_range_slider_mld)
def update_plot_mld(sel_month, col_range):
    plot = hv.QuadMesh(ds_mld.sel(month=sel_month), kdims=["xt_ocean", "yt_ocean"], 
                       vdims=hv.Dimension('mld', range=col_range))
    plot.opts(cmap=cmap_mld)
    plot.opts(title='Mixed-Layer Depth during ' + calendar.month_name[sel_month], framewise=True, colorbar=True, 
              clabel="m", width=900, height=250, tools=['hover'], ylabel='Latitude (°N)', 
              xlabel='Longitude (°E)', bgcolor='Silver', toolbar="above",
              hooks=[set_inactive_tool])
    return plot

@pn.depends(sel_month=month_slider_mld)
def update_cross_mld(x, sel_month):
    longitud = closest_lon(x or (initial_lon))

    # BATHYMETRY CROSS-SECTION
    # There's no sense in doing a QuadMesh for a MLD, because it's either there or it's not,
    # but it cannot have a range of values. We'll therefore plot an Area to show the extent.
    
    # First we generate the bathymetry of the ocean floor, so we can see how deep are the
    # MLDs with respect to the ocean floor
    bathy = hv.Area(bathymetry.sel(xt_ocean=closest_lon(x or initial_lon)), vdims="depth", 
                    label="Bathymetry")
    bathy.opts(framewise=True, invert_yaxis=True)

    # EXPERIMENT MLD CROSS-SECTION
    mixed = hv.Area(ds_mld.sel(xt_ocean=closest_lon(x or initial_lon))[month_slider_mld.value - 1], 
                    vdims="mld", label="Experiment")
    mixed.opts(framewise=True, invert_yaxis=True)

    # CONTROL MLD CROSS-SECTION
    mixed_c = hv.Area(ds_mld_c.sel(xt_ocean=closest_lon(x or initial_lon))[month_slider_mld.value - 1],
                      vdims="mld", label="Control")
    mixed_c.opts(framewise=True, invert_yaxis=True, color="LimeGreen", alpha=0.75)

    # We plot them all one on top of the other, so we can see where the MLDs overlap or not
    mld_lay = bathy * mixed * mixed_c
    mld_lay.opts(title='Mixed-Layer Depth cross-section at %.1f°E'% longitud, ylabel='Depth (m)', 
                 xlabel='Latitude (°N)', width=900, height=450, padding=(0, (0,0.05)), legend_position='bottom_left',
                 toolbar='above')

    return mld_lay
    
# Watch for changes on the widgets' values to update the plots
@pn.depends(sel_month=month_slider_mld)
def update_sealevel(x, sel_month):
    longitud = closest_lon(x or (initial_lon))

    # SEA-LEVEL CROSS-SECTION
    sl = hv.Area(ds_sealevel.sel(xt_ocean=closest_lon(x or initial_lon))[month_slider_mld.value - 1], 
                    vdims="sea_level", label="Experiment")
    sl.opts(framewise=True, color="Crimson")

    # CONTROL SEA-LEVEL CROSS-SECTION
    sl_c = hv.Area(ds_sealevel_c.sel(xt_ocean=closest_lon(x or initial_lon))[month_slider_mld.value - 1],
                      vdims="sea_level", label="Control")
    sl_c.opts(framewise=True, color="LimeGreen", alpha=0.75)

    sl_lay = sl * sl_c
    sl_lay.opts(title='Sea-Level cross-section at %.1f°E'% longitud, ylabel='Depth (m)', 
                 xlabel='Latitude (°N)', width=900, height=450, padding=(0, (0.05,0)), legend_position='bottom_right',
                 toolbar='above')

    return sl_lay

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# LONGITUDE LINE
vline_mld = hv.DynamicMap(lambda x: hv.VLine(x or initial_lon), streams=[posx_mld])
vline_mld.opts(color="Crimson", line_dash='dashed')

# U ANTARCTIC PLOT
plot_mld = hv.DynamicMap(update_plot_mld)

# U CROSS-SECTION
cross_mld = hv.DynamicMap(update_cross_mld, streams=[posx_mld])

# SSH CROSS-SECTION
sl_mld = hv.DynamicMap(update_sealevel, streams=[posx_mld])

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# Build the layout
pn.Column(pn.Row(pn.Spacer(width=50), month_slider_mld), 
          pn.Row(pn.Spacer(width=50), colorbar_range_slider_mld),
          pn.Spacer(height=5),
          plot_mld * vline_mld,
          cross_mld,
          sl_mld)

We could still go beyond: we've averaged the MLD by month, so now we don't know how's the inter-annual variability. Couldn't we do one of our time series for the MLD to look at that? Well, we'd need to select an area of interest in the ocean to look at the MLD there — it's pointless to average over the whole S.O. We'll just select the area we want on the bird's-eye view plot and generate the time-series for that.

In [ ]:
hv.extension('bokeh')

colorbar_range_mld = (0, ds_mld.max().values.max())
cmap_mld = "cmo.deep_r"
initial_lon = -68.5

# Stream to follow the pointer over the upper plot
posx_mld = hv.streams.PointerX()
# We'll add a new kind of Stream: on the bird's-eye view plot we'll add a selection box tool from Bokeh,
# so the user can drag and select the area of interest. The stream will then provide the coordinates of 
# the diagonal enclosing box: ((bottom-left X, bottom-left Y), (top-right X, top-right Y))
bounds_mld = hv.streams.BoundsXY()

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS

# Widget to select the MONTH
month_slider_mld = pn.widgets.DiscreteSlider(name='Month', options=ds_mld.month.values.tolist(), 
                                             value=ds_mld.month.values[0], width=500)

# Widget to select the COLORBAR RANGE
colorbar_range_slider_mld = pn.widgets.RangeSlider(name='Colorbar Range', start=0, 
                                                    end=colorbar_range_mld[1], # As weird as it looks, it's WNTBD
                                                    value=colorbar_range_mld, step=100, width=500)

# Widget to compute the MLD TIME-SERIES
# We'll display a text-box to show the coordinates for which we're computing the time series.
# It'll not only inform, but allow us to copy-paste the coordinates if we discover and area of interest.
mld_selection_text = pn.widgets.TextInput(value='Select area to compute time-series...', width=750)

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS's LINKS
# Watch for changes on the widgets' values to update the plots

@pn.depends(sel_month=month_slider_mld,
            col_range=colorbar_range_slider_mld)
def update_plot_mld(sel_month, col_range):
    plot = hv.QuadMesh(ds_mld.sel(month=sel_month), kdims=["xt_ocean", "yt_ocean"], 
                       vdims=hv.Dimension('mld', range=col_range))
    plot.opts(cmap=cmap_mld)
    plot.opts(title='Mixed-Layer Depth during ' + calendar.month_name[sel_month], framewise=True, colorbar=True, 
              clabel="m", width=900, height=250, tools=['hover', 'box_select'], ylabel='Latitude (°N)', 
              xlabel='Longitude (°E)', bgcolor='Silver', toolbar="above", active_tools=['box_select'],
              hooks=[set_inactive_tool])
    return plot

@pn.depends(sel_month=month_slider_mld)
def update_cross_mld(x, sel_month):
    longitud = closest_lon(x or (initial_lon))

    # BATHYMETRY CROSS-SECTION
    bathy = hv.Area(bathymetry.sel(xt_ocean=closest_lon(x or initial_lon)), vdims="depth", 
                    label="Bathymetry")
    bathy.opts(framewise=True, invert_yaxis=True)

    # MLD CROSS-SECTION
    mixed = hv.Area(ds_mld.sel(xt_ocean=closest_lon(x or initial_lon))[month_slider_mld.value - 1], 
                    vdims="mld", label="Experiment")
    mixed.opts(framewise=True, invert_yaxis=True)

    # CONTROL MLD CROSS-SECTION
    mixed_c = hv.Area(ds_mld_c.sel(xt_ocean=closest_lon(x or initial_lon))[month_slider_mld.value - 1],
                      vdims="mld", label="Control")
    mixed_c.opts(framewise=True, invert_yaxis=True, color="LimeGreen", alpha=0.75)

    mld_lay = bathy * mixed * mixed_c
    mld_lay.opts(title='Mixed-Layer Depth cross-section at %.1f°E'% longitud, ylabel='Depth (m)', 
                 xlabel='Latitude (°N)', width=900, height=450, padding=(0, (0,0.05)), legend_position='bottom_left',
                 toolbar='above')

    return mld_lay

@pn.depends(sel_month=month_slider_mld)
def update_sealevel(x, sel_month):
    longitud = closest_lon(x or (initial_lon))

    # SEA-LEVEL CROSS-SECTION
    sl = hv.Area(ds_sealevel.sel(xt_ocean=closest_lon(x or initial_lon))[month_slider_mld.value - 1], 
                    vdims="sea_level", label="Gold")
    sl.opts(framewise=True, color="Crimson")

    # CONTROL SEA-LEVEL CROSS-SECTION
    sl_c = hv.Area(ds_sealevel_c.sel(xt_ocean=closest_lon(x or initial_lon))[month_slider_mld.value - 1],
                      vdims="sea_level", label="Ctrl-9091")
    sl_c.opts(framewise=True, color="LimeGreen", alpha=0.75)

    sl_lay = sl * sl_c
    sl_lay.opts(title='Sea-Level cross-section at %.1f°E'% longitud, ylabel='Depth (m)', 
                 xlabel='Latitude (°N)', width=900, height=450, padding=(0, (0.05,0)), legend_position='bottom_right',
                 toolbar='above')

    return sl_lay

# Subscriber to update the information text and trigger the plotting of the time-series.
# This function will be called with the selection box values, in a different way from anything we've seen thus far.
def update_bounds_mld(bounds):
    if bounds is None:
        mld_selection_text.value = 'Select area to compute time-series'
    else:
        # We need to apply the same procedure we use to find the closest longitude from the widget's value
        # to find the closes latitude
        selection = '(' + str(closest_lat(bounds[1])) + ', '+  str(closest_lat(bounds[3])) + \
                    ', ' + str(closest_lon(bounds[0])) + ', '+  str(closest_lon(bounds[2])) + ')'
        mld_selection_text.value = "Experiment's MLD time-series for coordinates: " + selection + " as (lats, lons)"
    
# Watch for changes in the information text to update the time-series
@pn.depends(mld_selection_text)
def update_time_series(texto):
    # We need to check that a selection has been made. A "cheap way" of doing this is to simply 
    # check what's written on the widget: if it starts by 'Expe' then a selection has been made
    if texto[0:4] == "Expe":
        bounds = bounds_mld.bounds
        dsm.mld.loc[:, closest_lat(bounds[1]):closest_lat(bounds[3]), closest_lon(bounds[0]):closest_lon(bounds[2])].\
                mean(['yt_ocean', 'xt_ocean']).plot(size=7.7)
        ax = plt.gca()
        ax.set_ylim(ax.get_ylim()[::-1])
    
    curr_fig=plt.gcf()  
    # Remove the margins around the plot
    curr_fig.tight_layout()
    # Prevent pyplot from showing the plot (it would otherwise appear twice on-screen)
    plt.close(curr_fig) 
    return curr_fig
    
# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# LONGITUDE LINE
vline_mld = hv.DynamicMap(lambda x: hv.VLine(x or initial_lon), streams=[posx_mld])
vline_mld.opts(color="Crimson", line_dash='dashed')

# U ANTARCTIC PLOT
plot_mld = hv.DynamicMap(update_plot_mld)
# We'll take a different approach to tie the selection-box Stream to the bird's-eye view Antarctic Plot.
# We don't want to pass it as a 'streams' argument to the DynamicMap, because then the function
# update_plot_mld would need to receive the values from the Selection Box (otherwise there would be an error).
# Just receiving values to then do nothing with them is not very "clean" programming technique.
# Instead, we'll manually set the tie, specifying the plot we want as the source of the Stream.
bounds_mld.source = plot_mld
# Then, we inform the Stream of which function to call when there are changes in the Stream
bounds_mld.add_subscriber(update_bounds_mld)
# update_bounds_mld will update the Text Widget, which in turn will cause the calculation of the time-series

# U CROSS-SECTION
cross_mld = hv.DynamicMap(update_cross_mld, streams=[posx_mld])

# U CROSS-SECTION
sl_mld = hv.DynamicMap(update_sealevel, streams=[posx_mld])

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# Build the layout
pn.Column(pn.Row(pn.Spacer(width=50), month_slider_mld), 
          pn.Row(pn.Spacer(width=50), colorbar_range_slider_mld),
          pn.Spacer(height=5),
          plot_mld * vline_mld,
          cross_mld,
          sl_mld,
          pn.Spacer(height=5),
          pn.Row(pn.Spacer(width=45), mld_selection_text),
          update_time_series) # As we did with the quiver plots before, we can just pass a function to panel
                              # that will be automatically updated as necessary

Now, just select an interesting month like October, select an area to investigate with Bokeh's selection tool, and watch how the time-series is plotted at the bottom. 

This code is already displaying enough information to keep on extending it, so we'll stop this example here. As you can see, this is like a lego: we have the building blocks and we can make things as complicated as we want.

# 2. COMPARISON OF U-VELOCITIES

We'll now investigate u-velocities in the S.O., looking at a control run, then an experiment run, and then finally at the differences between both. Plotting all of them together is necessary because, when dealing with velocities, it's difficult to interpret the differences unless you can see the original data: has the current reversed direction, or simple slowed down?

In this case, we'll enable the option of averaging currents within a depth-range and within a month-range. We'll also expand our control of the colorbar by adding the option to use a discrete colorbar instead of a continuous one, which can sometimes help in better differentiating what's happening. And, since we're bothering with it, we'll also provide a way to adjust the number of bins of the discrete colorbar, because... why not?

In [ ]:
# Experiment U-velocity through time, averaged per month
ds_acc = dsm.u.loc[:,:,-78:-45,:]
ds_acc = ds_acc.groupby('time.month').mean('time')
# Ctrl U-velocity averaged through time, averaged per month
ds_acc_c = dsm_c.u.loc[:,:,-78:-45,:]
ds_acc_c = ds_acc_c.groupby('time.month').mean('time')

# Differences
ds_acc_dif = ds_acc - ds_acc_c

In [ ]:
hv.extension('bokeh')

# The ctrl & experiment plots will use one colorbar range, and the difference will need a separate one
colorbar_range_acc_avg = (-0.25, 0.25)
colorbar_range_acc_dif = (-0.05, 0.05)

cmap_acc_avg = "cmo.curl"
cmap_acc_dif = "RdBu_r"

initial_lon = -68.5

# Stream to follow the pointer over the bird's-eye view plots
posx_acc_avg = hv.streams.PointerX()

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS

# Widget to select the MONTH
month_slider_acc_avg = pn.widgets.IntRangeSlider(name='Month Range', start=1, end=12,
                                                 value=(1, 12), step=1, width=250)

# Widget to select the DEPTH
depth_range_acc_avg = pn.widgets.IntRangeSlider(name='Depth Range', start=0,
                                             end=np.int(np.rint(ds_acc.st_ocean.values[-1])),
                                             value=(0, 100), step=10, width=500)

# Widget to select the COLORBAR RANGE
colorbar_slider_acc_avg = pn.widgets.FloatSlider(name='Colorbar Range', start=0, end=1, step=0.05, 
                                             value=colorbar_range_acc_avg[1], width=370)

# Widget to activate a DISCRETE COLORBAR (instead of a continuous one)
discrete_colorbar_toggle_acc_avg = pn.widgets.Toggle(name='Discrete Colors', width=100)

# Widget to select the NUMBER of BINS of the colorbar
colorbins_slider_acc_avg = pn.widgets.IntSlider(name='Colorbar # Bins', start=5, end=30, step=1, value=15,
                                            width=250)

# Widget to select the COLORBAR RANGE for the differences
colorbar_slider_acc_dif = pn.widgets.FloatSlider(name='Colorbar Range', start=0, end=0.1, step=0.01, 
                                                 value=colorbar_range_acc_dif[1], width=370)

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS's LINKS
        
# Attach a link to the widget value, so it automatically refreshes the horizontal depth lines upon change.
# Since now depth is a range, we'll need 2 horizontal bars to show the depth range being averaged.
@pn.depends(depth=depth_range_acc_avg)
def update_hline_acc_avg_upper(depth):
    return hv.HLine(depth[0])

@pn.depends(depth=depth_range_acc_avg)
def update_hline_acc_avg_lower(depth):
    return hv.HLine(depth[1])


# EXPERIMENT
@pn.depends(sel_month=month_slider_acc_avg,
            depth=depth_range_acc_avg, 
            col_range=colorbar_slider_acc_avg,
            discrete_colors=discrete_colorbar_toggle_acc_avg,
            bin_count=colorbins_slider_acc_avg)
def update_plot_acc_avg(sel_month, depth, col_range, discrete_colors, bin_count):
    # Average data over the time- & depth-ranges selected
    plot = hv.QuadMesh(ds_acc.loc[sel_month[0]:sel_month[1], depth[0]:depth[1]].mean(['month','st_ocean']), 
                       kdims=["xu_ocean", "yu_ocean"], vdims=hv.Dimension('u', range=(-col_range, col_range)))
    # Decide which colormap to use, and the # of bins
    plot.opts(cmap=plt.cm.get_cmap(cmap_acc_avg, bin_count) if discrete_colors else cmap_acc_avg)
    # The np.in() call is necessary due to a minor bug in the widget (soon to be fixed).
    # We need to force an integer to prevent an error while looking up the month's name
    plot.opts(title=f'EXPERIMENT U-velocity between {depth[0]:d}–{depth[1]:d} (m) depth, '
              f'{calendar.month_name[np.int(sel_month[0])]}–{calendar.month_name[np.int(sel_month[1])]}', 
              framewise=True, colorbar=True, 
              clabel="m/s", width=900, height=250, tools=['hover'], ylabel='Latitude (°N)', xlabel='Longitude (°E)', 
              bgcolor='Silver', toolbar="above", hooks=[set_inactive_tool], active_tools=['box_zoom'])
        
    return plot

@pn.depends(sel_month=month_slider_acc_avg,
            col_range=colorbar_slider_acc_avg,
            discrete_colors=discrete_colorbar_toggle_acc_avg,
            bin_count=colorbins_slider_acc_avg)
def update_cross_acc_avg(x, sel_month, col_range, discrete_colors, bin_count):
    longitud = closest_lon_u(x or (initial_lon))
    
    plot = hv.QuadMesh(ds_acc.sel(month=slice(sel_month[0],sel_month[1]), xu_ocean=longitud).mean('month'), 
                       kdims=["yu_ocean", "st_ocean"], 
                       vdims=hv.Dimension('u', range=(-col_range, col_range)))
    plot.opts(cmap=plt.cm.get_cmap(cmap_acc_avg, bin_count) if discrete_colors else cmap_acc_avg)
    plot.opts(title='EXPERIMENT U-velocity cross-section at %d°E'% (longitud), framewise=True, invert_yaxis=True, 
              colorbar=True, clabel="m/s", bgcolor='Silver', tools=['hover'], width=900, height=450,
              ylabel='Depth (m)', xlabel='Latitude (°N)', toolbar="above", hooks=[set_inactive_tool],
              active_tools=['box_zoom'])
    return plot


# CONTROL
@pn.depends(sel_month=month_slider_acc_avg,
            depth=depth_range_acc_avg, 
            col_range=colorbar_slider_acc_avg,
            discrete_colors=discrete_colorbar_toggle_acc_avg,
            bin_count=colorbins_slider_acc_avg)
def update_plot_acc_c(sel_month, depth, col_range, discrete_colors, bin_count):
    plot = hv.QuadMesh(ds_acc_c.loc[sel_month[0]:sel_month[1], depth[0]:depth[1]].mean(['month','st_ocean']), 
                       kdims=["xu_ocean", "yu_ocean"], vdims=hv.Dimension('u', range=(-col_range, col_range)))
    plot.opts(cmap=plt.cm.get_cmap(cmap_acc_avg, bin_count) if discrete_colors else cmap_acc_avg)
    plot.opts(title=f'CONTROL U-velocity between {depth[0]:d}–{depth[1]:d} (m) depth, '
              f'{calendar.month_name[np.int(sel_month[0])]}–{calendar.month_name[np.int(sel_month[1])]}', 
              framewise=True, colorbar=True, 
              clabel="m/s", width=900, height=250, tools=['hover'], ylabel='Latitude (°N)', xlabel='Longitude (°E)', 
              bgcolor='Silver', toolbar="above", hooks=[set_inactive_tool], active_tools=['box_zoom'])
        
    return plot

@pn.depends(sel_month=month_slider_acc_avg,
            col_range=colorbar_slider_acc_dif,
            discrete_colors=discrete_colorbar_toggle_acc_avg,
            bin_count=colorbins_slider_acc_avg)
def update_cross_acc_c(x, sel_month, col_range, discrete_colors, bin_count):
    longitud = closest_lon_u(x or (initial_lon))
    
    plot = hv.QuadMesh(ds_acc_c.sel(month=slice(sel_month[0],sel_month[1]), xu_ocean=longitud).mean('month'), 
                       kdims=["yu_ocean", "st_ocean"], 
                       vdims=hv.Dimension('u', range=(-col_range, col_range)))
    plot.opts(cmap=plt.cm.get_cmap(cmap_acc_avg, bin_count) if discrete_colors else cmap_acc_avg)
    plot.opts(title='CONTROL U-velocity cross-section at %d°E'% (longitud), framewise=True, invert_yaxis=True, 
              colorbar=True, clabel="m/s", bgcolor='Silver', tools=['hover'], width=900, height=450,
              ylabel='Depth (m)', xlabel='Latitude (°N)', toolbar="above", hooks=[set_inactive_tool],
              active_tools=['box_zoom'])
    return plot


# DIFFERENCES
@pn.depends(sel_month=month_slider_acc_avg,
            depth=depth_range_acc_avg,  
            col_range=colorbar_slider_acc_dif,
            discrete_colors=discrete_colorbar_toggle_acc_avg,
            bin_count=colorbins_slider_acc_avg)
def update_plot_acc_dif(sel_month, depth, col_range, discrete_colors, bin_count):
    plot = hv.QuadMesh(ds_acc_dif.loc[sel_month[0]:sel_month[1], depth[0]:depth[1]].mean(['month','st_ocean']), 
                       kdims=["xu_ocean", "yu_ocean"], vdims=hv.Dimension('u', range=(-col_range, col_range)))
    plot.opts(cmap=plt.cm.get_cmap(cmap_acc_dif, bin_count) if discrete_colors else cmap_acc_dif)
    plot.opts(title=f'DiFFERENCES in U-velocity between {depth[0]:d}–{depth[1]:d} (m) depth, '
              f'{calendar.month_name[np.int(sel_month[0])]}–{calendar.month_name[np.int(sel_month[1])]}', 
              framewise=True, colorbar=True, 
              clabel="m/s", width=900, height=250, tools=['hover'], ylabel='Latitude (°N)', xlabel='Longitude (°E)', 
              bgcolor='Silver', toolbar="above", hooks=[set_inactive_tool], active_tools=['box_zoom'])
        
    return plot

@pn.depends(sel_month=month_slider_acc_avg, 
            col_range=colorbar_slider_acc_dif,
            discrete_colors=discrete_colorbar_toggle_acc_avg,
            bin_count=colorbins_slider_acc_avg)
def update_cross_acc_dif(x, sel_month, col_range, discrete_colors, bin_count):
    longitud = closest_lon_u(x or (initial_lon))
    
    plot = hv.QuadMesh(ds_acc_dif.sel(month=slice(sel_month[0],sel_month[1]), xu_ocean=longitud).mean('month'), 
                       kdims=["yu_ocean", "st_ocean"], 
                       vdims=hv.Dimension('u', range=(-col_range, col_range)))
    plot.opts(cmap=plt.cm.get_cmap(cmap_acc_dif, bin_count) if discrete_colors else cmap_acc_dif)
    plot.opts(title='DiFFERENCES in U-velocity cross-section at %d°E'% (longitud), framewise=True, invert_yaxis=True, 
              colorbar=True, clabel="m/s", bgcolor='Silver', tools=['hover'], width=900, height=450,
              ylabel='Depth (m)', xlabel='Latitude (°N)', toolbar="above", hooks=[set_inactive_tool],
              active_tools=['box_zoom'])
    return plot

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# LONGITUDE LINE

# Just as an example, we can also create the vertical line in place instead of through a function
vline_acc_avg = hv.DynamicMap(lambda x: hv.VLine(x or initial_lon), streams=[posx_acc_avg])
vline_acc_avg.opts(color="RoyalBlue", line_dash='dashed')

# DEPTH LINES
hline_acc_avg_upper = hv.DynamicMap(update_hline_acc_avg_upper)
hline_acc_avg_upper.opts(color="RoyalBlue", line_dash='dashed')

hline_acc_avg_lower = hv.DynamicMap(update_hline_acc_avg_lower)
hline_acc_avg_lower.opts(color="RoyalBlue", line_dash='dashed')

# All plots will update together, following the cursor over whichever plot we're on, by tying them to the same stream.
# That way we can ensure that the information being shown is consistent amongst plots.
# U ANTARCTIC PLOT
plot_acc_avg = hv.DynamicMap(update_plot_acc_avg)
# U CROSS-SECTION
cross_acc_avg = hv.DynamicMap(update_cross_acc_avg, streams=[posx_acc_avg])

# CTRL U ANTARCTIC PLOT
plot_acc_c = hv.DynamicMap(update_plot_acc_c)
# CTRL U CROSS-SECTION
cross_acc_c = hv.DynamicMap(update_cross_acc_c, streams=[posx_acc_avg])

# DIFFERENCES U ANTARCTIC PLOT
plot_acc_dif = hv.DynamicMap(update_plot_acc_dif)
# DIFFERENCES U CROSS-SECTION
cross_acc_dif = hv.DynamicMap(update_cross_acc_dif, streams=[posx_acc_avg])

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# Build the layout
pn.Column(pn.Row(pn.Spacer(width=50), depth_range_acc_avg, month_slider_acc_avg), 
          pn.Row(pn.Spacer(width=50), colorbar_slider_acc_avg, discrete_colorbar_toggle_acc_avg, 
                 pn.Spacer(width=2), colorbins_slider_acc_avg),
          pn.Spacer(height=5),
          plot_acc_avg * vline_acc_avg,
          cross_acc_avg * hline_acc_avg_upper * hline_acc_avg_lower,
          plot_acc_c * vline_acc_avg,
          cross_acc_c * hline_acc_avg_upper * hline_acc_avg_lower,
          plot_acc_dif * vline_acc_avg,
          cross_acc_dif * hline_acc_avg_upper * hline_acc_avg_lower,
          pn.Row(pn.Spacer(width=50), colorbar_slider_acc_dif))

We can hardly get more sophisticated than this. By now, you have all the code & tools you need to create great, interactive panels to explore your data. Share them with the community! ^_^